In [0]:
!pip install torch-geometric
!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
!pip install --upgrade torch-sparse
!pip install --upgrade torch-cluster

     |████████████████████████████████| 143kB 3.4MB/s 
     |████████████████████████████████| 348kB 7.7MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.4.2-cp36-none-any.whl size=224447 sha256=336ca78d0e9eb4ed843ce2045179fa3c84fa5bf74a4d8100aa6c1e919bf5d8a2
  Stored in directory: /root/.cache/pip/wheels/0f/9d/a3/2de0c1fb436ec76e2942b4216b820589d40bc48395995c3076
  Created wheel for plyfile: filename=plyfile-0.7.1-cp36-none-any.whl size=32828 sha256=a3c9454d300eedd9c529b9f0b8e31ea8335a66a8e7816dff70b45545d470c08b
  Stored in directory: /root/.cache/pip/wheels/d6/0d/bf/6d603d81b98604d2ecfd5e99d4ab7c9af664fd5285ab82bbb0
Successfully built torch-geometric plyfile
Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 10.6MB 352kB/s 
     |████████████████████████████████| 11.5MB 71.5MB/s 
Requirement already up-to-date: torch-sparse

In [0]:
import torch
import torch.nn as n
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as f
import random
import torch.optim as op

In [0]:
import torch_geometric.nn as tg
import torch_geometric.data as Data
import torch_geometric.utils as ut

In [0]:
from torch_geometric.datasets import TUDataset

In [0]:
dataset1=TUDataset(root="dataset1/",name="ENZYMES",use_node_attr=True)

Extracting dataset1/ENZYMES/ENZYMES.zip
Processing...
Done!


In [0]:
data=dataset1[0]

In [0]:
data

Data(edge_index=[2, 168], x=[37, 21], y=[1])

In [0]:
e1=ut.add_remaining_self_loops(data.edge_index,edge_weight=2*torch.ones(data.edge_index.shape[1]),fill_value=3)

In [0]:
e1

(tensor([[ 0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
           3,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,
           7,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
          12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16,
          16, 16, 17, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 20, 20,
          21, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25,
          25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28,
          28, 28, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 32,
          32, 32, 32, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 35, 35, 35,
          35, 35, 36, 36, 36, 36,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
          12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
          30, 31, 32, 33, 34, 35, 36],
         [ 1,  2,  3,  0,  2,  3, 24, 27,  0,  1,  3, 27, 2

In [0]:
# 

In [0]:
data.edge_index.shape[1]

168

In [0]:
# edge_weight >> is weight that has to be given to edge embeddings of shape total number of edges of graph
# fill_value >> weight that has to be given to self loops that is multiplied with self edges

In [0]:
# fill value weight can also be used to give amount of weight given to self node embeddings

In [0]:
from torch_scatter import scatter_add

In [0]:
deg=scatter_add(e1[1],e1[0][0],dim=0,dim_size=37)

In [0]:
deg

tensor([ 9., 13., 13., 15., 11., 13., 11., 13.,  9., 13., 11., 11., 13., 11.,
        11., 11., 11., 11.,  9.,  9., 15., 13., 11., 11., 11., 13., 13., 13.,
        15., 17., 13.,  9., 11., 15., 15., 13., 11.])

In [0]:
e1[0][0]

tensor([ 0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
         3,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,
         7,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
        12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16,
        16, 16, 17, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 20, 20,
        21, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25,
        25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28,
        28, 28, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 32,
        32, 32, 32, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 35, 35, 35,
        35, 35, 36, 36, 36, 36,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
        12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
        30, 31, 32, 33, 34, 35, 36])

In [0]:
e1

(tensor([[ 0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
           3,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,
           7,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
          12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16,
          16, 16, 17, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 20, 20,
          21, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25,
          25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28,
          28, 28, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 32,
          32, 32, 32, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 35, 35, 35,
          35, 35, 36, 36, 36, 36,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
          12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
          30, 31, 32, 33, 34, 35, 36],
         [ 1,  2,  3,  0,  2,  3, 24, 27,  0,  1,  3, 27, 2

In [0]:
e1=ut.add_remaining_self_loops(data.edge_index,edge_weight=(0.5)*torch.ones(data.edge_index.shape[1]),fill_value=3)

In [0]:
e1[1]

tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 

In [0]:
deg=scatter_add(e1[1],e1[0][0],dim=0,dim_size=37)

In [0]:
deg #so degree is calculated using weights with weights being multiplied to node features

tensor([4.5000, 5.5000, 5.5000, 6.0000, 5.0000, 5.5000, 5.0000, 5.5000, 4.5000,
        5.5000, 5.0000, 5.0000, 5.5000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        4.5000, 4.5000, 6.0000, 5.5000, 5.0000, 5.0000, 5.0000, 5.5000, 5.5000,
        5.5000, 6.0000, 6.5000, 5.5000, 4.5000, 5.0000, 6.0000, 6.0000, 5.5000,
        5.0000])